Imported Libaries

In [1]:
import os
root_dir = r'c:\Users\jf79\Cloned Repos\Footfall-Repo'
os.chdir(root_dir)

import pandas as pd

import myfuncs.myfuncs as mf
import repofuncs.footfallfuncs as ff

mf.read_directory()

import warnings
warnings.filterwarnings('ignore')

Your Current Directory is: c:\Users\jf79\Cloned Repos\Footfall-Repo
Files in: ['.git', 'Footfall.venv', 'Footfall_Scripts', 'myfuncs', 'README.md', 'repofuncs', 'requirements.txt', 'Spend_Scripts']



In [2]:
cwd = r'C:/Users/jf79/OneDrive - Office Shared Service/Documents/H&F Analysis/Footfall and Spend Analysis/Footfall Data'
mf.read_directory(cwd)

Directory being read is: C:/Users/jf79/OneDrive - Office Shared Service/Documents/H&F Analysis/Footfall and Spend Analysis/Footfall Data
Files in: ['Area Based', 'Hex Based', 'LSOA Based', 'Mapping']



In [3]:
footfall_2024_data = pd.read_csv(f'{cwd}/Hex Based/Footfall Counts/hex_3hourly_counts_2024.csv')
hex_to_borough_data = pd.read_csv(f'{cwd}/Hex Based/Footfall Counts/BT Hex ID to Borough Lookup Table.csv')
relevant_hexes_data = pd.read_csv(f'{cwd}/Hex Based/Relevant Hexes/Relevant Hexes.csv')
bespoke_areas_data = pd.read_csv(f'{cwd}/Area Based/bespoke_3hourly_counts_2022-05-01_2025-02-09.csv')

In [4]:
values_to_keep = [
    "Sands End","Fulham Town","Parsons Green & Sandford","Palace & Hurlingham","Munster",
    "Walham Green","Lillie","Fulham Reach","West Kensington","Hammersmith Broadway",
    "Avonmore","Ravenscourt","Brook Green","Grove","Addison","Wendell Park",
    "Coningham","Shepherd's Bush Green","Wormholt","White City","College Park & Old Oak"
]

bespoke_areas_data = pd.read_csv(f'{cwd}/Area Based/bespoke_3hourly_counts_2022-05-01_2025-02-09.csv')
ward_footfall = bespoke_areas_data.copy()

ward_footfall = ward_footfall[ward_footfall['name'].isin(values_to_keep)]
ward_footfall['hours'] = ward_footfall['hours'].str.strip("'")
ward_footfall.rename(
    columns={
        'hours':'time_indicator',
    },
    inplace=True
)
ward_footfall


,bespoke_area_id,name,count_date,time_indicator,resident,visitor,worker,ave_loyalty_percentage,ave_dwell_time
1953166,257,Sands End,2022-05-01,00-03,957,179,89,20.26,109.56
1953167,257,Sands End,2022-05-01,03-06,12538,4633,969,2.16,104.36
1953168,257,Sands End,2022-05-01,06-09,13383,5149,1224,3.42,139.48
1953169,257,Sands End,2022-05-01,09-12,15997,8379,1660,6.97,100.04
1953170,257,Sands End,2022-05-01,12-15,14451,10695,1972,5.95,88.25
...,...,...,...,...,...,...,...,...,...
2122626,277,College Park & Old Oak,2025-02-09,09-12,31155,13968,5345,7.69,87.92
2122627,277,College Park & Old Oak,2025-02-09,12-15,30906,21957,5956,9.36,73.75
2122628,277,College Park & Old Oak,2025-02-09,15-18,30295,23090,6002,8.35,74.16
2122629,277,College Park & Old Oak,2025-02-09,18-21,31449,17293,4932,6.38,83.96


In [5]:
lon_footfall_2024 = footfall_2024_data.copy()
relevant_hexes = relevant_hexes_data.copy()
hex_to_borough = hex_to_borough_data.copy()

borough_footfall_2024 = pd.merge(
    lon_footfall_2024,
    hex_to_borough,
    right_on='Hex_ID',
    left_on='hex_id',
    how='left'
)

borough_footfall_2024 = borough_footfall_2024.groupby(['borough_name','count_date','time_indicator']).agg(
    resident = ('resident','sum'),
    worker = ('worker','sum'),
    visitor = ('visitor','sum')
).reset_index()

hf_footfall_2024 = pd.merge(
    relevant_hexes_data,
    lon_footfall_2024,
    left_on='Hex_ID',
    right_on='hex_id',
    how='left'
)

In [6]:
typical_day_averages = ff.typical_footfall(
    hf_footfall_2024, '2024-09-01', '2024-12-31',
    footfall_type=['residents','workers','visitors'],
    day_night='day_night',
    primary_key='hex_id'
)

for i in range(len(typical_day_averages)):
    if 'averages' in typical_day_averages[i].columns.to_list():
        display(typical_day_averages[i].sort_values(by='averages', ascending=False))
    elif 'daytime_mean' in typical_day_averages[i].columns.to_list():
        display(typical_day_averages[i].sort_values(by='daytime_mean', ascending=False))
    else:
        display(typical_day_averages[i])

Calculating typical daily footfall...
For Weedays and Weekends and Weekly averages...


Aggregating footfall data...
Missing kwargs: {'std', 'time_indicator'}
These args will be set to default values

Applying features...
Features applied.


Detecting anomalies...
1570 anomalies have been detected.
Anomalies have been flagged and corrected.


Detecting anomalies...
624 anomalies have been detected.
Anomalies have been flagged and corrected.


Detecting anomalies...
1326 anomalies have been detected.
Anomalies have been flagged and corrected.

Footfall Data Aggregated.


Transforming to daynight...


,year,hex_id,daytime_mean,nighttime_mean
114,2024,11281238,58335.934426,25920.934426
92,2024,11271229,25573.081967,12119.016393
91,2024,11271227,23823.024590,15934.622951
107,2024,11281224,16791.352459,7567.672131
180,2024,11341220,16015.663934,10195.647541
...,...,...,...,...
87,2024,11271219,46.901639,34.303279
66,2024,11251245,44.524590,12.819672
86,2024,11271217,18.516393,18.090164
67,2024,11251247,17.327869,2.000000


,year,week_name,hex_id,daytime_mean,nighttime_mean
114,2024,Weekday,11281238,53020.252874,25622.126437
92,2024,Weekday,11271229,28261.333333,12592.643678
91,2024,Weekday,11271227,25200.000000,16295.229885
107,2024,Weekday,11281224,19902.448276,7853.574713
180,2024,Weekday,11341220,16500.758621,10395.287356
...,...,...,...,...,...
66,2024,Weekday,11251245,59.034483,13.632184
87,2024,Weekday,11271219,48.172414,34.264368
67,2024,Weekday,11251247,21.494253,1.850575
86,2024,Weekday,11271217,18.218391,19.195402


,year,week_name,hex_id,daytime_mean,nighttime_mean
319,2024,Weekend,11281238,71549.200000,26663.685714
296,2024,Weekend,11271227,20400.257143,15038.257143
297,2024,Weekend,11271229,18890.857143,10941.714286
385,2024,Weekend,11341220,14809.857143,9699.400000
278,2024,Weekend,11261228,12400.228571,6971.485714
...,...,...,...,...,...
292,2024,Weekend,11271219,43.742857,34.400000
291,2024,Weekend,11271217,19.257143,15.342857
271,2024,Weekend,11251245,8.457143,10.800000
272,2024,Weekend,11251247,6.971429,2.371429


In [7]:
mf.export_to_csv(typical_day_averages[0])

Exproting  to CSV...
@ C:\Users\jf79\OneDrive - Office Shared Service\Documents\H&F Analysis\Python CSV Repositry\.csv

Successfully exported  to CSV
